In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_movie_budgets():
    """
    Scrapea la lista de películas de 'https://www.the-numbers.com/movie/budgets/all',
    incluyendo la fecha de lanzamiento (Release Date), título (Movie), presupuesto (Budget),
    recaudación en EE.UU. (Domestic Gross) y recaudación mundial (Worldwide Gross).
    """
    base_url = "https://www.the-numbers.com/movie/budgets/all"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/112.0.0.0 Safari/537.36"
        )
    }
    
    all_data = []
    page = 1
    
    while True:
        offset = (page - 1) * 100
        if offset == 0:
            url = base_url
        else:
            url = f"{base_url}/{offset+1}"
        
        print(f"Scrapeando página {page}: {url}")
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Status {response.status_code}. Se detiene la extracción.")
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Busca la primera tabla que encuentre
        table = soup.find("table")
        if not table:
            print("No se encontró la tabla en la página.")
            break
        
        # Selecciona todas las filas <tr> de esa tabla
        table_rows = table.find_all("tr")
        
        # Si la tabla tiene una fila de cabecera, se salta
        if len(table_rows) <= 1:
            print("No se encontraron más datos.")
            break
        
        for row in table_rows[1:]:
            cols = row.find_all("td")
            if len(cols) < 6:
                continue
            
            release_date    = cols[1].get_text(strip=True)
            movie           = cols[2].get_text(strip=True)
            budget          = cols[3].get_text(strip=True).replace('\xa0','')
            domestic_gross  = cols[4].get_text(strip=True).replace('\xa0','')
            worldwide_gross = cols[5].get_text(strip=True).replace('\xa0','')
            
            # Limpieza adicional (quitar $ y comas)
            budget = budget.replace('$', '').replace(',', '')
            domestic_gross = domestic_gross.replace('$', '').replace(',', '')
            worldwide_gross = worldwide_gross.replace('$', '').replace(',', '')
            
            all_data.append({
                'Release Date': release_date,
                'Movie': movie,
                'Budget': budget,
                'Domestic Gross': domestic_gross,
                'Worldwide Gross': worldwide_gross
            })
        
        page += 1

    df = pd.DataFrame(all_data)
    
    # Filtrar filas con fechas desconocidas o incorrectas
    df = df[df['Release Date'].str.match(r'^\w{3} \d{1,2}, \d{4}$')]
    
    # Convertir la columna "Release Date" al formato "YYYY-MM-DD"
    df['Release Date'] = pd.to_datetime(df['Release Date'], format='%b %d, %Y').dt.strftime('%Y-%m-%d')
    
    return df


if __name__ == "__main__":
    df_budgets = scrape_movie_budgets()
    print(df_budgets.head())
    
    df_budgets.to_csv('../data/the_numbers_movie_budgets-gross.csv', index=False)
    print(f"Guardado CSV con {len(df_budgets)} registros.")

Scrapeando página 1: https://www.the-numbers.com/movie/budgets/all
Scrapeando página 2: https://www.the-numbers.com/movie/budgets/all/101
Scrapeando página 3: https://www.the-numbers.com/movie/budgets/all/201
Scrapeando página 4: https://www.the-numbers.com/movie/budgets/all/301
Scrapeando página 5: https://www.the-numbers.com/movie/budgets/all/401
Scrapeando página 6: https://www.the-numbers.com/movie/budgets/all/501
Scrapeando página 7: https://www.the-numbers.com/movie/budgets/all/601
Scrapeando página 8: https://www.the-numbers.com/movie/budgets/all/701
Scrapeando página 9: https://www.the-numbers.com/movie/budgets/all/801
Scrapeando página 10: https://www.the-numbers.com/movie/budgets/all/901
Scrapeando página 11: https://www.the-numbers.com/movie/budgets/all/1001
Scrapeando página 12: https://www.the-numbers.com/movie/budgets/all/1101
Scrapeando página 13: https://www.the-numbers.com/movie/budgets/all/1201
Scrapeando página 14: https://www.the-numbers.com/movie/budgets/all/1301
S